<a href="https://colab.research.google.com/github/IgorVaryvoda/Image-captioning/blob/main/Auto_alt_tags_Sirv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image alt tags are important for both accessibility and SEO. But writing alt tags and adding them to images can be quite a chore. What if there was a way to automate this process?

Luckily, image recognition AI models like BLIP have come a long way, and services like Replicate allow us to use them via simple API calls. Let's leverage the power of AI and Sirv to automatically add alt tags to your images.

[Read the full article here ](https://sirv.com/help/articles/auto-alt-tags-ai)

Save a dopy of the notebook and follow these steps (press the "Run cell" buttons near each code block):

# 1. Install Prerequisites



In [1]:
!pip install replicate
# Download the demo images file (VM filesystem isn't persistent)
!wget -q https://sirv.sirv.com/website/screenshots/auto-alt-tags/images.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for replicate: filename=replicate-0.5.1-py3-none-any.whl size=14795 sha256=b0e9ad5e60bf151ceaa44a1dbf275bea5c75387e71d2f671548f9eb61e6c4c7e
  Stored in directory: /root/.cache/pip/wheels/ba/4e/82/3a8bb76d45a5f81c79a1ff0eaa1dd43b69822f5d622280b6eb
Successfully built replicate


#2. Add API keys

Get a Replicate API key here - <a href="https://replicate.com/signin?next=/docs">here</a>
Get Sirv keys by [following these instructions](https://sirv.com/help/articles/auto-alt-tags-ai#get-sirv-rest-api-keys)

In [17]:
#@title 2.3 Add your API keys here
#@markdown Replicate key

replicate_api_key = '' #@param {type:"string"}
#@markdown Sirv keys
sirv_id = '' #@param {type: "string"}
sirv_secret = '' #@param {type: "string"}
#@markdown Choose the model. BLIP is faster, but less detailed, BLIP2 is more accurate, but way more resource intensive. 
# Uncheck the checkbox below to use BLIP instead of BLIP2
use_blip2 = True #@param {type: "boolean"}

# Add your images 
Add images to /content/images.txt

Learn how to get them here - https://sirv.com/help/articles/auto-alt-tags-ai#adding-image-alt-tags-in-bulk

# Run the code

In [18]:
import replicate
import os
import json
import requests
import sys
from urllib.parse import urlparse
os.environ["REPLICATE_API_TOKEN"] = replicate_api_key

def get_token():
  # Get Sirv auth token. API reference - https://apidocs.sirv.com/#connect-to-sirv-api
  payload = {
    'clientId': sirv_id,
    'clientSecret': sirv_secret
  }
  headers = {'content-type': 'application/json', "Origin": "*"}
  response = requests.request('POST', 'https://api.sirv.com/v2/token', data=json.dumps(payload), headers=headers)
  global token
  if response:
    token = response.json()['token']
  else:
    print('There is an error in your credentials. Please check if your ID and secret are correct.')
    sys.exit()

def get_description(image):
  global output
  if (use_blip2):
    model = replicate.models.get("salesforce/blip-2")
    version = model.versions.get("4b32258c42e9efd4288bb9910bc532a69727f9acd26aa08e175713a0a857a608")
    # Get the image description
    try:
      inputs = {
          'image': image,
          'caption': True,
          'question': False,
          'use_nucleus_sampling': False,
          'temperature': 1,
      }
      output = version.predict(**inputs)
    except Exception as e: print(e)
  else:
    model = replicate.models.get("salesforce/blip")
    version = model.versions.get("2e1dddc8621f72155f24cf2e0adbde548458d3cab9f00c0139eea840d0ac4746")
    try:
        inputs = {
            # Input image to query or caption
            'image': image,
            'task': "image_captioning",
        }
        output = version.predict(**inputs)
    except Exception as e: print(e)

def update_meta(image, output, token):
  '''
  Update the image description in Sirv.
  API reference - https://apidocs.sirv.com/#set-meta-description
  '''
  description = '{"description": "'+ output.capitalize() + '"}'
  params = {"filename": urlparse(image).path.replace('\n', '')}
  headers = {
    'content-type': "application/json",
    "Origin": "*",
    'authorization': 'Bearer %s' % token
  }

  try:
    print('Updating the image description in Sirv')
    response = requests.request('POST', 'https://api.sirv.com/v2/files/meta/description', data=description.encode('utf-8'), headers=headers, params=params)
    if response.status_code == 200:
      print('✅success')
    elif response.status_code == 401: #token expired, get new one
      get_token()
  except requests.exceptions.RequestException as e:
    raise SystemExit(e)

#We're opening the txt file and looping through each image in the file
get_token()
with open('images.txt') as f:
  for image in f:
    #Replicate stuff
    get_description(image)
    print(f"Working on file: {image}, caption:{output}")
    if (len(output) == 0):
      print("No description detected.")
    else:
        update_meta(image, output, token)


Working on file: https://cdn.earthroulette.com/help-ukraine/protest.jpg
, caption:a crowd of people holding signs and flags
Updating the image description in Sirv
success
Working on file: https://cdn.earthroulette.com/help-ukraine/dontfundwar.jpg
, caption:the logo for danone on a black background
Updating the image description in Sirv
success
Working on file: https://cdn.earthroulette.com/help-ukraine/refugees.jpg
, caption:two women hugging in front of a tent
Updating the image description in Sirv
success
Working on file: https://cdn.earthroulette.com/help-ukraine/army.jpg
, caption:ukrainian soldiers in camouflage uniforms standing next to each other
Updating the image description in Sirv
success
Working on file: https://cdn.earthroulette.com/help-ukraine/FMWRQvhXsAESunV.jpg
, caption:the flag of ukraine is fluttering in the wind
Updating the image description in Sirv
success
Working on file: https://cdn.earthroulette.com/help-ukraine/petition.webp, caption:a hand writing on a piece